In [1]:
%reset -f
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import profiling as prof
from nltk.util import ngrams
from collections import Counter
import itertools
from numpy import dot
from numpy.linalg import norm
from math import sqrt

data9 = prof.load_data("data/capture20110817.binetflow.txt")
data10 = prof.load_data("data/capture20110818-2.binetflow.txt")
data11 = prof.load_data("data/capture20110818.binetflow.txt")
data12 = prof.load_data("data/capture20110819.binetflow.txt")
pdata9 = prof.pre_process(data9,9)
pdata10 = prof.pre_process(data10,10)
pdata11 = prof.pre_process(data11,11)
pdata12 = prof.pre_process(data12,12)

We preprocess the data for scenario 9 to 12 and discretize the protocol feature the same way we did as the previous tasks. Scenario 9 will be the 'training' scenario, in which we will choose a suitable threshold for the distance calculation in the other scenarios. This threshold separates the new predicted infected hosts from normal hosts. First an infected source/host is selected from scenario 9, in this case with the most netflows in the dataset to train as best as possible. We then calculate all 3-grams in the flows from this host and count the occurences, resulting in a profile for this host. Instead of using a timed sliding-window, the unique hosts are grouped and profiles are created from counts of 3-grams. The choice for 3-grams is based on the fact that the possible combinations of 2-grams are quite low (only 4 combinations) and there are not enough flows originating from the unique hosts for using 4-grams. 

For each unique host we calculate the 3-grams profile the same way as we did for the selected host, and additionaly we calculate the cosine distance in relation to the selected host. At the end of this iteration the distance between the initially selected host and all unique hosts is determined. We must choose a threshold that separates the infected and normal hosts as correctyly as possible and additionaly optimizes the number of newly identified infected hosts. Part of the hosts in the dataset are labeled as infected or normal, while there is a part that is not labeled yet and thus to be identified. A grid search is applied for a threshold of 0 to 0.25 (cosine distance). The results show that with a threshold of 0.18 there are 12 (out of 19 unidentified) new infections identified at cost of a couple of true positives and false positives, dropping the accuracy to 62.5%. The false positives indicate hosts that are labeled as infected while they are not. It is thus important to keep this number as low as possible in comparison to the number of newly identified infected hosts.

With this chosen threshold we can do the same for the other three scenarios. We find that the results with this threshold do not deviate much for the other scenarios. We identify zero, seven and six newly identified infections to respectively scenario 9, 10 and 11, at cost of one, one and five false positives. We can thus conclude that by profiling using n-grams new infected hosts can be detected. However, the more the threshold is on 'edge', the more this results in an increase in false positives. This is a trade-off someone should be willing to make (or not).

In [2]:
scenarios = [9, 10, 11, 12]
grid_search = pd.DataFrame(columns=["Scenario", "Threshold","Unknown","TP","FP","FN","TN","Accuracy","NEW_INFECTIONS"])

for scenario in scenarios:
    if scenario == 9:
        pdata = pdata9.copy()
    elif scenario == 10:
        pdata = pdata10.copy()
    elif scenario == 11:
        pdata = pdata11.copy()
    elif scenario == 12:
        pdata = pdata12.copy() 

    # Feature to use, including its optimal binsize, and discretized to be used for n-grams
    feature = "Protocol"
    nbins = 3

    discretized_data = pd.DataFrame()
    discretized_data["StartTime"] = pdata["StartTime"].copy()
    discretized_data["SourceAddress"] = pdata["SourceAddress"].copy()
    discretized_data["DestinationAddress"] = pdata["DestinationAddress"].copy()
    pdata["Protocol"] = prof.encode_feature(pdata["Protocol"])
    discretized_data[feature], binsedges_infected = prof.discretize_feature(pdata, feature, nbins, "kmeans")

    # Select source with most entries in given dataset to deliver most reliable n-gram profile
    # Determine list of all possible n-grams for selected host
    # This is the 3-gram profile of this host
    selected_source = prof.select_infected_host(pdata,scenario)
    filtered_source = discretized_data.loc[discretized_data.SourceAddress == selected_source]

    ngram = 3
    all_n_grams = list(itertools.product(*[['0','1','2'],['0','1','2'],['0','1','2']]))

    n_grams = pd.Index(list(ngrams(filtered_source[feature].astype(str),ngram)))
    n_grams_counts = [0]*len(all_n_grams)

    for i in range(len(n_grams.value_counts())):
        n_gram = n_grams.value_counts()
        index_all_n_grams = all_n_grams.index(n_gram.index[i])
        n_grams_counts[index_all_n_grams] = n_gram[i]

    # Group netflows by source address (host) because we are interested in modeling per-host behavior
    unique_sources = pdata.groupby(["SourceAddress"]).size().reset_index()

    # Calculate the distance for every other source compared to the selected source/host
    distances = [100]*len(unique_sources)
    n_gram_count_list = list()

    for i in range(len(unique_sources)):
        source = unique_sources.iloc[i]

        filtered = discretized_data.loc[discretized_data.SourceAddress == source.SourceAddress]
        n_grams_source = pd.Index(list(ngrams(filtered[feature].astype(str),ngram)))
        n_grams_source_counts = [0]*len(all_n_grams)

        for j in range(len(n_grams_source.value_counts())):
            n_gram = n_grams_source.value_counts()
            index_all_n_grams = all_n_grams.index(n_gram.index[j])
            n_grams_source_counts[index_all_n_grams] = n_gram[j]


        # Cosine distance between selected host and other hosts
        if np.sum(n_grams_source_counts) == 0:
            distances[i] = 1
        else:
            distances[i] = 1-dot(n_grams_counts, n_grams_source_counts)/(norm(n_grams_counts)*norm(n_grams_source_counts))


    # Create an overview of the results including the original label and predicted label
    results = pd.DataFrame(columns=["SourceAddress","Cosine_Distance","Label","Prediction"])

    for i in range(len(distances)):
        distance = distances[i]
        source = unique_sources.SourceAddress.iloc[i]
        infected = prof.is_infected(source, scenario)
        normal = prof.is_normal(source)
        label = -1

        if infected == True:
            label = 1
        elif normal == True:
            label = 0

        results = results.append({"SourceAddress": source, "Cosine_Distance": distance, "Label": label, "Prediction": ''}, ignore_index=True)
    
    # For each threshold ranging from 0 to 0.25, in steps of 0.01 we check the results to be able to choose
    # good threshold. This is only based on 'training' scenario 9, the other sets will use the threshold 
    # that has been chosen optimal in scenario 9
    thresholds = np.arange(0,0.25,0.01)
    for threshold in thresholds:
        results.Prediction = results.Cosine_Distance.apply(lambda x: 1 if x <= threshold else 0)

        TP, FP, FN, TN, new_infections, unknown = 0, 0, 0, 0, 0, 0

        for i in range(len(results)):
            row = results.iloc[i]
            if row.Label == row.Prediction and row.Label == 1:
                TP += 1
            elif row.Label == 0 and row.Prediction == 1:
                FP += 1
            elif row.Label == 0 and row.Prediction == 0:
                TN += 1
            elif row.Label == 1 and row.Prediction == 0:
                FN += 1
                
            # Number of unlabeled hosts
            if row.Label == -1:
                unknown += 1
                
            # Newly identified infected host
            if row.Label == -1 and row.Prediction == 1:
                new_infections += 1
                
        accuracy = (TP+TN)/(TP+FP+FN+TN)
                
        grid_search = grid_search.append({"Scenario": scenario, "Threshold": threshold, "Unknown": unknown, "TP": TP, "FP": FP, "FN": FN, "TN": TN, "Accuracy": accuracy,"NEW_INFECTIONS": new_infections}, ignore_index=True)

# Uncheck print statement to see all results of results
pd.set_option('display.max_rows', None)
print(grid_search)
        
# Manual evaluation of results of grid search delivered an optimal threshold of 0.18
# For scenario 9 this threshold delivers 12 new infections at the cost of a couple false positives
# For the other scenarios it didn't really matter which threshold to choose
filter_threshold = grid_search.loc[grid_search.Threshold == 0.18]
filter_threshold.head()

    Scenario  Threshold  Unknown   TP   FP    FN   TN  Accuracy  \
0        9.0       0.00     19.0  0.0  0.0  10.0  6.0  0.375000   
1        9.0       0.01     19.0  2.0  0.0   8.0  6.0  0.500000   
2        9.0       0.02     19.0  5.0  0.0   5.0  6.0  0.687500   
3        9.0       0.03     19.0  6.0  0.0   4.0  6.0  0.750000   
4        9.0       0.04     19.0  7.0  1.0   3.0  5.0  0.750000   
5        9.0       0.05     19.0  9.0  1.0   1.0  5.0  0.875000   
6        9.0       0.06     19.0  9.0  1.0   1.0  5.0  0.875000   
7        9.0       0.07     19.0  9.0  1.0   1.0  5.0  0.875000   
8        9.0       0.08     19.0  9.0  1.0   1.0  5.0  0.875000   
9        9.0       0.09     19.0  9.0  2.0   1.0  4.0  0.812500   
10       9.0       0.10     19.0  9.0  3.0   1.0  3.0  0.750000   
11       9.0       0.11     19.0  9.0  3.0   1.0  3.0  0.750000   
12       9.0       0.12     19.0  9.0  3.0   1.0  3.0  0.750000   
13       9.0       0.13     19.0  9.0  3.0   1.0  3.0  0.75000

,Scenario,Threshold,Unknown,TP,FP,FN,TN,Accuracy,NEW_INFECTIONS
18,9.0,0.18,19.0,9.0,5.0,1.0,1.0,0.625000,12.0
43,10.0,0.18,9.0,2.0,1.0,1.0,5.0,0.777778,0.0
68,11.0,0.18,19.0,3.0,1.0,0.0,5.0,0.888889,7.0
93,12.0,0.18,14.0,3.0,5.0,0.0,1.0,0.444444,6.0


### Fingerprinting

In [4]:
import operator
from math import nan

# For each scenario pick the infected host with the most unique n-gram.
# Then check which n-gram does not occur in the non-infected host.
# Pick top k n-grams that satisfy above criteria, check for other hosts

scenarios = [9, 10, 11, 12]
pd_results = pd.DataFrame()
result_scenario = []
result_ip = []
result_TP = []
result_FP = []
result_TN = []
result_FN = []
result_newinf = []
result_acc = []
result_used_ngrams = []
for scenario in scenarios:
    if scenario == 9:
        pdata = pdata9.copy()
    elif scenario == 10:
        pdata = pdata10.copy()
    elif scenario == 11:
        pdata = pdata11.copy()
    elif scenario == 12:
        pdata = pdata12.copy() 

    # Feature to use, including its optimal binsize, and discretized to be used for n-grams
    feature = "Protocol"
    nbins = 3

    discretized_data = pd.DataFrame()
    discretized_data["StartTime"] = pdata["StartTime"].copy()
    discretized_data["SourceAddress"] = pdata["SourceAddress"].copy()
    discretized_data["DestinationAddress"] = pdata["DestinationAddress"].copy()
    pdata["Protocol"] = prof.encode_feature(pdata["Protocol"])
    discretized_data[feature], binsedges_infected = prof.discretize_feature(pdata, feature, nbins, "kmeans")

    # Select source with most entries in given dataset to deliver most reliable n-gram profile
    selected_sources = prof.select_all_infected_host(pdata,scenario)
    # ONLY FOR TEST LOOP OVER ALL INFECTED IPS     
    for selected_source in selected_sources:
        result_scenario.append(scenario)
        result_ip.append(selected_source)

        # selected_source = prof.select_infected_host(pdata,scenario)
        filtered_source = discretized_data.loc[discretized_data.SourceAddress == selected_source]

        ngram = 3
        all_n_grams = list(itertools.product(*[['0','1','2'],['0','1','2'],['0','1','2']]))

        n_grams = pd.Index(list(ngrams(filtered_source[feature].astype(str),ngram)))
        n_grams_counts = [0]*len(all_n_grams)
        for i in range(len(n_grams.value_counts())):
            n_gram = n_grams.value_counts()
            index_all_n_grams = all_n_grams.index(n_gram.index[i])
            n_grams_counts[index_all_n_grams] = n_gram[i]

        # Select all benign traffic(Non-infected)
        non_infected_host = prof.select_non_infected_host(pdata)
        unique_sources_benign = pd.DataFrame()
        unique_sources_benign['SourceAddress'] = non_infected_host

        # Sum all counts of all benign host(if n-gram == 0, does not occur in benign data)
        sum_ngrams_benign = [0]*len(all_n_grams)
        for i in range(len(unique_sources_benign)):
            source = unique_sources_benign.iloc[i]

            filtered = discretized_data.loc[discretized_data.SourceAddress == source.SourceAddress]
            n_grams_source = pd.Index(list(ngrams(filtered[feature].astype(str),ngram)))
            n_grams_source_counts = [0]*len(all_n_grams)

            for j in range(len(n_grams_source.value_counts())):
                n_gram = n_grams_source.value_counts()
                index_all_n_grams = all_n_grams.index(n_gram.index[j])
                n_grams_source_counts[index_all_n_grams] = n_gram[j]

            # Check which n-gram does NOT occur in benign data
            sum_ngrams_benign = np.add(sum_ngrams_benign, n_grams_source_counts)

        # Pick all ngrams that does not occur in benign traffic     
        n_grams_not_in_benign = np.array(sum_ngrams_benign)
        n_grams_not_in_benign = np.where(sum_ngrams_benign == 0)[0]

        # Pick all ngrams that occur in infected traffic     
        n_grams_in_infected = np.array(n_grams_counts)
        n_grams_in_infected = np.where(n_grams_in_infected > 0)[0]

        # Pick the ngrams that does not occur in benign but occur in infected data     
        possible_fingerprints = np.intersect1d(n_grams_in_infected, n_grams_not_in_benign)
        result_used_ngrams.append(possible_fingerprints)

        ### PART 2 (CHECK WITH THE NGRAMS FIND ABOVE IF ANY (NEW) INFECTED HOSTS COULD BE DETECTED) ####    
        
        # Group netflows by source address (host) because we are interested in modeling per-host behavior
        unique_sources = pdata.groupby(["SourceAddress"]).size().reset_index()

        finger_print_exists = [100]*len(unique_sources)

        # Check if fingerprint is contained in the data of the host
        for i in range(len(unique_sources)):
            source = unique_sources.iloc[i]

            filtered = discretized_data.loc[discretized_data.SourceAddress == source.SourceAddress]
            n_grams_source = pd.Index(list(ngrams(filtered[feature].astype(str),ngram)))
            n_grams_source_counts = [0]*len(all_n_grams)

            for j in range(len(n_grams_source.value_counts())):
                n_gram = n_grams_source.value_counts()
                index_all_n_grams = all_n_grams.index(n_gram.index[j])
                n_grams_source_counts[index_all_n_grams] = n_gram[j]

            # Pick the indexes of the ngram that occur          
            n_grams_in_source = np.array(n_grams_source_counts)
            n_grams_in_source = np.where(n_grams_in_source > 0)[0]
            print(n_grams_in_source)

            # For each possible fingerprint check if it exists in the data
            for fingerprint in possible_fingerprints:
                # Raise alarm         
                if fingerprint in n_grams_in_source:
                    finger_print_exists[i] = 1

        # # Create an overview of the results including the original label and predicted label
        results = pd.DataFrame(columns=["SourceAddress","Fingerprint_exists","Label","Prediction"])

        for i in range(len(finger_print_exists)):
            finger_print = finger_print_exists[i]
            source = unique_sources.SourceAddress.iloc[i]
            infected = prof.is_infected(source, scenario)
            normal = prof.is_normal(source)
            label = -1

            if infected == True:
                label = 1
            elif normal == True:
                label = 0

            results = results.append({"SourceAddress": source, "Fingerprint_exists": finger_print, "Label": label, "Prediction": finger_print}, ignore_index=True)
        TP, FP, FN, TN, new_infections, unknown = 0, 0, 0, 0, 0, 0

        for i in range(len(results)):
            row = results.iloc[i]
            if row.Label == row.Prediction and row.Label == 1:
                TP += 1
            elif row.Label == 0 and row.Prediction == 1:
                FP += 1
            elif row.Label == 0 and row.Prediction == 0:
                TN += 1
            elif row.Label == 1 and row.Prediction == 0:
                FN += 1

            # Number of unlabeled hosts
            if row.Label == -1:
                unknown += 1

            # Newly identified infected host
            if row.Label == -1 and row.Prediction == 1:
                new_infections += 1
        if (TP+FP+FN+TN) == 0:
            accuracy = nan
        else: accuracy = (TP+TN)/(TP+FP+FN+TN)
        result_TP.append(TP)
        result_FP.append(FP)
        result_TN.append(TN)
        result_FN.append(FN)
        result_newinf.append(new_infections)
        result_acc.append(accuracy)

pd_results['Scenario'] = result_scenario
pd_results['Infected IP used'] = result_ip
pd_results['Used Ngrams'] = result_used_ngrams
pd_results['TP'] = result_TP
pd_results['FP'] = result_FP
pd_results['TN'] = result_TN
pd_results['FN'] = result_FN
pd_results['New infected'] = result_newinf
pd_results['Accuracy'] = result_acc
pd_results.head(50)


[26]
[26]
[]
[26]
[ 8 11 13 14 16 17 21 22 23 25 26]
[26]
[]
[26]
[ 0  1  3  4  5  8  9 10 11 12 13 14 16 17 19 20 21 22 23 24 25 26]
[ 4  5  8 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25 26]
[ 8 11 13 14 16 17 21 22 23 25 26]
[26]
[ 5  7  8 10 11 12 13 14 15 16 17 20 21 22 23 24 25 26]
[13 14 16 17 22 23 25 26]
[13 14 16 17 22 23 25 26]
[13 14 16 17 22 23 25 26]
[ 8 13 14 16 17 20 22 23 24 25 26]
[ 4 10 13 14 16 17 21 22 23 25 26]
[13 14 16 17 22 23 25 26]
[ 4 10 12 13 14 15 16 17 19 22 23 24 25 26]
[13 14 16 17 22 23 25 26]
[26]
[26]
[26]
[26]
[26]
[]
[]
[]
[]
[]
[26]
[ 0  2  8 24]
[26]
[ 8 11 12 14 16 17 20 21 22 23 24 25 26]
[26]
[26]
[]
[26]
[ 8 11 13 14 16 17 21 22 23 25 26]
[26]
[]
[26]
[ 0  1  3  4  5  8  9 10 11 12 13 14 16 17 19 20 21 22 23 24 25 26]
[ 4  5  8 10 11 12 13 14 15 16 17 19 20 21 22 23 24 25 26]
[ 8 11 13 14 16 17 21 22 23 25 26]
[26]
[ 5  7  8 10 11 12 13 14 15 16 17 20 21 22 23 24 25 26]
[13 14 16 17 22 23 25 26]
[13 14 16 17 22 23 25 26]
[13 14 16 17 22 23 25

[ 0  2  6  7  8  9 11 13 14 16 17 18 20 21 22 23 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[ 0  2  7  8 16 17 18 20 22 23 24 25 26]
[ 0  2  8 15 16 17 18 20 23 24 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[ 0  2  7  8 16 17 18 23 24 25 26]
[ 0  2  6  8 16 17 18 20 23 24 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[26]
[26]
[26]
[26]
[13]
[]
[]
[26]
[0]
[26]
[ 7  8 11 12 13 14 16 17 19 20 21 22 23 24 25 26]
[26]
[26]
[26]
[ 4 10 13 14 16 17 21 22 23 25 26]
[26]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]
[ 0  2  7  8 16 17 18 23 24 25 26]
[ 0  2  6  7  8  9 11 13 14 16 17 18 20 21 22 23 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[ 0  2  7  8 16 17 18 20 22 23 24 25 26]
[ 0  2  8 15 16 17 18 20 23 24 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[ 0  2  8 16 17 18 23 24 25 26]
[ 0  2  7  8 16 17 18 23 24 25 26]
[ 0  2  6  8 16 17 18 20 23 24 25 26]
[ 0  2  8 16 17 18 23 24 25

,Scenario,Infected IP used,Used Ngrams,TP,FP,TN,FN,New infected,Accuracy
0,9,147.32.84.165,[15],3,0,0,0,0,1.0
1,9,147.32.84.191,"[7, 15]",3,0,0,0,0,1.0
2,9,147.32.84.192,[],0,0,0,0,0,NaN
3,9,147.32.84.193,[],0,0,0,0,0,NaN
4,9,147.32.84.204,[],0,0,0,0,0,NaN
5,9,147.32.84.205,[],0,0,0,0,0,NaN
6,9,147.32.84.206,[],0,0,0,0,0,NaN
7,9,147.32.84.207,[],0,0,0,0,0,NaN
8,9,147.32.84.208,[15],3,0,0,0,0,1.0
9,9,147.32.84.209,[],0,0,0,0,0,NaN


The profiles found by the "Botnet profiling task" are here used to detect new infections by fingerprinting. For each scenario, the infected host with the most entries in the dataset is selected since this host has the most reliable n-gram profile. The n-gram(s) that only exists in the data of the infected host and not in the n-grams of all benign data is then used to detect possible new infected hosts.

Above the results of the fingerprinting algorithm can be seen. ...
 
Compared to the botnet profiling task, ...


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0